In [2]:
!pip install tensorflow


   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/331.9 MB 1.3 MB/s eta 0:04:17
   ---------------------------------------- 1.0/331.9 MB 1.5 MB/s eta 0:03:37
   ---------------------------------------- 2.1/331.9 MB 2.4 MB/s eta 0:02:15
   ---------------------------------------- 2.6/331.9 MB 2.4 MB/s eta 0:02:18
   ---------------------------------------- 2.6/331.9 MB 2.4 MB/s eta 0:02:18
   ---------------------------------------- 3.1/331.9 MB 2.0 MB/s eta 0:02:44
   ---------------------------------------- 3.9/331.9 MB 2.2 MB/s eta 0:02:27
    --------------------------------------- 4.7/331.9 MB 2.4 MB/s eta 0:02:15
    --------------------------------------- 5.5/331.9 MB 2.6 MB/s eta 0:02:08
    --------------------------------------- 6.3/331.9 MB 2.6 MB/s eta 0:02:04
    --------------------------------------- 7.3/331.9 MB 2.8 MB/s eta 0:01:57

In [22]:
import tensorflow as tf

In [23]:
import tensorflow as tf
import os

# Define the path to your main dataset folder
data_dir = 'chest_xray'  # Replace with the actual path
batch_size = 32
image_height = 224
image_width = 224

# Create the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, 'train'),
    image_size=(image_height, image_width),
    batch_size=batch_size,
    shuffle=True
)

# Create the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, 'val'),
    image_size=(image_height, image_width),
    batch_size=batch_size
)

# Create the test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, 'test'),
    image_size=(image_height, image_width),
    batch_size=batch_size
)

# Print the class names
class_names = train_ds.class_names
print("Class Names:", class_names)

# You can optionally apply a normalization layer to the datasets
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Example of how to iterate through the dataset
# for images, labels in train_ds.take(1):
#     print(f"Batch shape (images): {images.shape}")
#     print(f"Batch shape (labels): {labels.shape}")

Found 5216 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 624 files belonging to 2 classes.
Class Names: ['NORMAL', 'PNEUMONIA']


In [24]:
# Assuming your dataset loading code has run and 'class_names' is defined
num_classes = len(class_names)

In [25]:
from tensorflow import keras
from keras import layers

 normalization layer to scale the pixel values

In [26]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

data augmentation to the training set to improve performance.

In [27]:
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
])

# Now you can use this pipeline to augment your training data
# (Or just add these layers directly to the beginning of your model)

.cache() keeps the images in memory after the first epoch, avoiding disk I/O.

.prefetch() overlaps the data preprocessing and model execution, which prevents your GPU from sitting idle while it waits for new data.

In [28]:
# The datasets are already normalized in the previous step,
# but let's add caching and prefetching for efficiency.
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

BUILDING THE MODEL.

In [29]:
# New way that follows best practices and removes the warning
model = keras.Sequential([
    keras.Input(shape=(image_height, image_width, 3)),  # The new Input layer
    layers.Conv2D(32, 3, activation='relu'),             # No input_shape needed here
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Optional: Print a summary of your model's architecture
model.summary()
epochs = 10 # You can adjust this value
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_21 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,218 (42.61 MB)

 Trainable params: 11,169,218 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 221s 1s/step - accuracy: 0.7429 - loss: 0.5777 - val_accuracy: 0.5000 - val_loss: 0.7657
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 177s 1s/step - accuracy: 0.8359 - loss: 0.3654 - val_accuracy: 0.8125 - val_loss: 0.4185
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.9367 - loss: 0.1661 - val_accuracy: 0.8125 - val_loss: 0.4690
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 196s 1s/step - accuracy: 0.9467 - loss: 0.1400 - val_accuracy: 0.8750 - val_loss: 0.2762
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.9536 - loss: 0.1270 - val_accuracy: 0.8750 - val_loss: 0.2275
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 180s 1s/step - accuracy: 0.9599 - loss: 0.1070 - val_accuracy: 0.6875 - val_loss: 0.5100
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.9599 - loss: 0.1057 - val_accuracy: 0.6875 - val_loss: 0.6292
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 151s 924ms/step - accuracy: 0.9651 - loss: 0.0989 - val_a

MODEL EVALUATION.

In [31]:
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - accuracy: 0.7580 - loss: 1.1405
Test Loss: 1.1405
Test Accuracy: 0.7580


In [32]:
model.save('my_model.keras')

In [4]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Define the variables to match your training setup
class_names = ['NORMAL', 'PNEUMONIA']
img_height = 224
img_width = 224

# Load the saved model (using the new .keras format)
model_path = 'my_model.keras'
loaded_model = tf.keras.models.load_model('my_model.keras')

# Load a new image for prediction
new_image_path = r'C:\Users\shafiiya shaik\Downloads\demo_img.jpg'
# Make sure to replace this with the actual path to your image
try:
    img = Image.open(r'C:\Users\shafiiya shaik\Downloads\demo_img.jpg').convert('RGB')
    img = img.resize((img_height, img_width))
    img_array = np.array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    
    # Preprocess the image (normalize it)
    # The preprocessing MUST match what you did during training
    # Cast the image data to float32 before normalizing
    img_array = tf.cast(img_array, tf.float32) / 255.0
    # Make the prediction
    predictions = loaded_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)

    # The rest of your prediction code runs here...

# Corrected print statement
    print(f"The image '{r'C:\Users\shafiiya shaik\Downloads\demo_img.jpg'}' most likely belongs to {predicted_class} with a {confidence:.2f}% confidence.")

except FileNotFoundError:
    print(f"Error: The image file was not found at {r'C:\Users\shafiiya shaik\Downloads\demo_img.jpg'}. Please check the path.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step
The image 'C:\Users\shafiiya shaik\Downloads\demo_img.jpg' most likely belongs to PNEUMONIA with a 73.11% confidence.
